In [5]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool
from datetime import datetime
import warnings
import joblib
import seaborn as sns
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import joblib

warnings.filterwarnings("ignore")

In [6]:
# # Paths to datasets
# train_path = "/kaggle/input/playground-series-s4e7/train.csv"
# test_path = "/kaggle/input/playground-series-s4e7/test.csv"

# Paths to datasets
train_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv"
test_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\test.csv"


In [7]:
def import_data(path, index_col=None):
    """Import data from a CSV file and optimize memory usage."""
    df = pd.read_csv(path, index_col=index_col)
    return reduce_mem_usage(df)

def reduce_mem_usage(df):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    for col in df.columns:
        col_type = df[col].dtype
        if isinstance(col_type, pd.IntervalDtype):
            continue

        if str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)  
        elif str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
    return df

def feature_engineering(df):
    """Feature engineering on the dataset."""
    # Binning age and converting to categorical labels instead of intervals
    age_bins = pd.cut(df['Age'], bins=7, labels=False)
    df['Age_Type'] = age_bins
    df['Vehicle_Age'] = df['Vehicle_Age'].astype('category').cat.codes
    df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category').cat.codes
    df['Previously_Insured'] = df['Previously_Insured'].astype('category').cat.codes

    df['Age_x_Vehicle_Age'] = df['Age_Type'] * df['Vehicle_Age']
    df['Age_x_Vehicle_Damage'] = df['Age_Type'] * df['Vehicle_Damage']
    df['Age_x_Previously_Insured'] = df['Age_Type'] * df['Previously_Insured']

    fac_pre = ['Policy_Sales_Channel', 'Vehicle_Damage', 'Annual_Premium', 'Vintage', 'Age_Type']
    col_pre = []
    for i in fac_pre:
        df['Previously_Insured_x_' + i] = pd.factorize(df['Previously_Insured'].astype(str) + df[i].astype(str))[0]
        col_pre.append('Previously_Insured_x_' + i)

    fac_pro = fac_pre[1:]
    col_pro = []
    for i in fac_pro:
        df['Policy_Sales_Channel_x_' + i] = pd.factorize(df['Policy_Sales_Channel'].astype(str) + df[i].astype(str))[0]
        col_pro.append('Policy_Sales_Channel_x_' + i)
    return df, col_pre, col_pro

def reduce_mem_usage_np(array):
    """Reduce memory usage of a numpy array by converting its dtype without losing significant precision."""
    for col in range(array.shape[1]):
        col_data = array[:, col]
        if np.issubdtype(col_data.dtype, np.floating):
            col_min = col_data.min()
            col_max = col_data.max()
            if col_min > np.finfo(np.float16).min and col_max < np.finfo(np.float16).max:
                array[:, col] = col_data.astype(np.float16)
            elif col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                array[:, col] = col_data.astype(np.float32)
        elif np.issubdtype(col_data.dtype, np.integer):
            col_min = col_data.min()
            col_max = col_data.max()
            if col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max:
                array[:, col] = col_data.astype(np.int8)
            elif col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max:
                array[:, col] = col_data.astype(np.int16)
            elif col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max:
                array[:, col] = col_data.astype(np.int32)
    return array

In [8]:
# Load and optimize data
train_df = import_data(train_path, index_col='id')
test_df = import_data(test_path, index_col='id')

train_df = train_df.sample(frac=0.005)
test_df = test_df.sample(frac=0.005)

# Combine train and test datasets for consistent transformation
full_df = pd.concat([train_df, test_df], axis=0)

In [9]:
# Convert columns to category type
less = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel']
for col in less:
    full_df[col] = full_df[col].astype('category')

# Apply feature engineering to the combined dataset
full_df, col_pre, col_pro = feature_engineering(full_df)

In [10]:
# Split back into train and test sets
train_df = full_df.iloc[:len(train_df), :]
test_df = full_df.iloc[len(train_df):, :]

# Split the training data into training and validation sets
X = train_df.drop('Response', axis=1)
y = train_df['Response']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [11]:
# Define the ColumnTransformer
coltrans = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, dtype=np.float32), ['Gender', 'Vehicle_Damage']),
        ('minmax', MinMaxScaler(), ['Age', 'Region_Code', 'Previously_Insured', 'Policy_Sales_Channel', 'Vintage']),
        ('ordinal', OrdinalEncoder(categories=[[0, 1, 2]], dtype=np.float32), ['Vehicle_Age']),
        ('robust', RobustScaler(), ['Annual_Premium']),
        ('standard', StandardScaler(), ['Age_Type', 'Age_x_Vehicle_Age', 'Age_x_Vehicle_Damage', 'Age_x_Previously_Insured']),
        ('standard_2', StandardScaler(), col_pre + col_pro),
    ],
    remainder='passthrough'  # Keeps columns not specified in transformers
)

# Fit the transformer on the training data and transform both training and validation sets
X_train = coltrans.fit_transform(X_train)
X_valid = coltrans.transform(X_valid)
test_df = coltrans.transform(test_df.drop('Response', axis=1))

# Reduce memory usage of transformed data
X_train = reduce_mem_usage_np(X_train)
X_valid = reduce_mem_usage_np(X_valid)
test_df = reduce_mem_usage_np(test_df)

gc.collect()

5013

In [12]:
# Calculate the ratio for scale_pos_weight
ratio = len(train_df[train_df['Response'] == 0]) / len(train_df[train_df['Response'] == 1])
class_weights = {0: 1, 1: ratio}

In [13]:
# Define LightGBM parameters
lgb_params = {
               'depth': 6,
               "eval_metric": "auc",
               "early_stopping_round": 50,
               "max_bin": 255,
               'num_leaves': 223,
               'learning_rate': 0.028095688623590447,
               'min_child_samples': 54,
               'subsample': 0.5395472919165504,
               'colsample_bytree': 0.547518064129546,
               'lambda_l1': 3.4444245446562,
               'lambda_l2': 2.87490408088595e-05,
               'scale_pos_weight': ratio,
               'early_stopping_rounds': 10,
               'device': 'gpu'
}

# Train LightGBM model
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

lgb = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=2500,
    valid_sets=[train_data, valid_data],
)

valid_preds = lgb.predict(X_valid, num_iteration=lgb.best_iteration)
auc_score = roc_auc_score(y_valid, valid_preds)

test_pred_lgb = lgb.predict(test_df, num_iteration=lgb.best_iteration)

gc.collect()

[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2343
[LightGBM] [Info] Number of data points in the train set: 46019, number of used features: 25
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2070 SUPER, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 20 dense feature groups (0.88 MB) transferred to GPU in 0.002779 secs. 1 sparse feature groups
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Start training from score 0.124253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improv

1058

In [14]:
# Define CatBoost parameters
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.05,
    'iterations': 5000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'task_type': 'GPU',  # Ensure your environment supports GPU
    'allow_writing_files': False,
    'verbose': 100,
    'class_weights': class_weights 
    # 'thread_count': -1
}

test_pool = Pool(test_df)
train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)
    
cbc = CatBoostClassifier(**cat_params)
cbc.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50, verbose=100)
    
valid_preds = cbc.predict_proba(X_valid)[:, 1]
auc_score = roc_auc_score(y_valid, valid_preds)

test_pred_cbc = cbc.predict_proba(test_pool)[:, 1]

gc.collect()

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8453719	best: 0.8453719 (0)	total: 360ms	remaining: 29m 57s
100:	test: 0.8621059	best: 0.8621059 (100)	total: 14.2s	remaining: 11m 27s
200:	test: 0.8671202	best: 0.8671796 (193)	total: 24.2s	remaining: 9m 37s
300:	test: 0.8681221	best: 0.8683334 (295)	total: 31.5s	remaining: 8m 11s
400:	test: 0.8693229	best: 0.8693229 (400)	total: 41.1s	remaining: 7m 51s
bestTest = 0.869322896
bestIteration = 400
Shrink model to first 401 iterations.


0

In [15]:
# Define XGBoost parameters
xgb_params = {
    'random_state': 512,
    'objective': "binary:logistic",
    'eval_metric': 'auc',
    'max_depth': 8,
    'min_child_weight': 12,
    'colsample_bytree': 0.5,
    'gamma': 0.2,
    'learning_rate': 0.09093568107192034,
    'subsample': 1.0,
    'reg_alpha': 0.0011852827097616767,
    'reg_lambda': 1.0735757602378362e-06,
    'max_bin': 197818,
    'scale_pos_weight': ratio,  # Adjust this based on your dataset
    'tree_method': 'hist',  # Ensure your environment supports GPU
    'device': 'cuda',  # Ensure your environment supports GPU
}

dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=3000,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        verbose_eval=100,
        early_stopping_rounds=10,
    )
    
valid_preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))
auc_score = roc_auc_score(y_valid, valid_preds)
    
dtest = xgb.DMatrix(test_df)
test_pred_xgb = model.predict(dtest, iteration_range=(0, model.best_iteration))

[0]	train-auc:0.86354	valid-auc:0.85240
[67]	train-auc:0.91498	valid-auc:0.86906


In [17]:
# Blending predictions with calculated weights
blended_preds = (test_pred_cbc + test_pred_lgb* test_pred_xgb)/3

# Reimport the test_df to get the original index
test_df = import_data(test_path, index_col='id')

# Create a submission DataFrame using the original test index
submission = pd.DataFrame({
    'id': test_df.index,
    'Response': blended_preds
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

ValueError: All arrays must be of the same length